In [ ]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Use CUDA</h2>

In [ ]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
import numpy as np
import pandas as pd
from scipy import optimize, special

F = np.array([[1.0000, 0.1000],
              [0, 1.0000]])

G = np.array([[0.0050],
              [0.1000]])

C = np.array([[1, 0]])

D = 0
Gain = np.array([16.0302, 5.6622])

L = np.array([[0.9902],
              [0.9892]])
safex = [26,31]

In [ ]:
k=14
inputs = np.random.normal(0, 1, k)
input_len=len(inputs)
X = np.zeros((2, input_len + 1))
Xe = np.zeros((2, input_len + 1))
e = np.zeros((2, input_len))
U = np.zeros((1,input_len))
Y = np.zeros((1,input_len))
r = np.zeros((1,input_len))
z = np.zeros((1,input_len))
S = np.zeros((input_len))
n1 = np.random.normal(0,0.01, [1, input_len])      
n2 = np.random.normal(0,0.001, [2, input_len])

In [ ]:
for i in range(input_len):
    e[:, i] = X[:, i] - Xe[:, i]
    U[:,i] = -(Gain @ Xe[:, i])
    X[:, i+1] = F @ X[:, i] + G @ U[:, i] + n2[:, i]
    Y[:,i] = C @ X[:, i] + n1[:,i]
    r[:,i] = Y[:,i] - C @ Xe[:, i]
    Xe[:, i+1] = F @ Xe[:, i] + G @ U[:,i] + L @ r[:,i]
covn_r=np.cov(r)

In [ ]:
attackK= [0, 0.04155838 , 0.63838859 , 1.75426837 ,2.44935936 , 3.6651983 , 5.16561739 , 6.95368618 , 9.11716362 ,11.67267658 ,14.29706254, 17.41085821  ,0.       ,   0.    ]
k=len(attackK)
attack2K = [-30.971675   ,-30.971675  ,  -1.11684483 ,-30.90514398, -28.29714574 , -28.61888554, -28.91105434, -29.17567371, -29.41469648, -29.21724115 , -28.49741602 , 30.971675,     0.     ,      0. ]
k1=len(attack2K)

In [ ]:
inputs = np.random.normal(0, 1, k)
input_len=len(inputs)
Xk = np.zeros((2, input_len + 1))
Xek = np.zeros((2, input_len + 1))
ek = np.zeros((2, input_len))
Uk = np.zeros((1,input_len))
Uak = np.zeros((1,input_len))
Yk = np.zeros((1,input_len))
rk = np.zeros((1,input_len))
covn_eK = np.zeros((2, 2))
covn_rK = np.zeros((2, 2))
zk = np.zeros((1,input_len))
Sk = np.zeros((input_len))
n1 = np.random.normal(0,0.01, [1, input_len])      
n2 = np.random.normal(0,0.001, [2,input_len])

In [ ]:
residuek=[]
mzk=[]
bias=4
for i in range(input_len):
    ek[:, i] = Xk[:, i] - Xek[:, i]
    Uk[:,i] = -(Gain @ Xek[:, i])
    Uak[:,i] = -(Gain @ Xek[:, i])+attack2K[i]
    Xk[:, i+1] = F @ Xk[:, i] + G @ Uak[:, i] + n2[:, i]
    Yk[:,i] = C @ Xk[:, i] + n1[:,i] + attackK[i]
    rk[:,i] = Yk[:,i] - C @ Xek[:, i]
    Xek[:, i+1] = F @ Xek[:, i] + G @ Uk[:,i] + L @ rk[:,i]
    zk[:,i] = rk[:,i].T * covn_r * rk[:,i]
    residuek.append(rk[:,i])
    mzk.append(zk[:,i])
    Sk[i] = max((Sk[i-1] + zk[:,i] - bias), 0)

In [ ]:
Sk

In [ ]:
m=np.mean(np.array(mzk))

In [ ]:
import math
def discrete_frame(Sk,N,th):
        delSi = 2*th/(2*N-1)
        Ek=np.zeros((len(Sk)))
        for i in range(len(Sk)):
            if Sk[i]<delSi[i]/2:
                Ek[i]=0
            elif Sk[i]>=th[i]:
                Ek[i]=N
            else :
                Ek[i]=math.ceil(math.floor(Sk[i]/(delSi[i]/2))/2)
        return Ek

In [ ]:
N=10

In [ ]:
residuek=np.absolute(np.array(residuek))

In [ ]:
r_mean=np.array([])
for i in range(len(residuek)):
    sums=0
    for j in range(i+1):
        sums+=residuek[j]
    mean=sums/(j+1)
    r_mean=np.append(r_mean,mean)

In [ ]:
r_mean

In [ ]:
np.count_nonzero(r_mean>0.01)

In [ ]:
import numpy as np
from scipy.special import erf
import scipy.special as special
from scipy import special, optimize
def markov_transition(N, th, bias):
    temp = np.zeros((N+1, N+1))
    for i in range(1,N+1):
        for j in range(1,N+1):
            if j == 1 & i <= N:
                temp[i, j] = Tx(1-i, th, bias, N)
            elif j > 1 & j < N+1 & i <= N:
                temp[i, j] = Px(j-i, th, bias, N)
            elif j == N+1 & i<=N:
                temp[i, j] = 1 - Tx(N-i, th, bias, N)
            elif i == N+1 & j < N+1:
                temp[i, j] = 0
            else:
                temp[i, j] = 1
    P = temp
    return P

def Px(x, th, bias, N):
    delSi = 2*th/(2*N-1)
    z1 = np.max(x*delSi + 0.5*delSi + bias, 0)
    p1 = special.gammainc(m/2, z1/2)
    z2 = np.max(x*delSi - 0.5*delSi + bias, 0)
    p2 = special.gammainc(m/2, z2/2)
    p = p1-p2
    return p
def Tx(x, th, bias, N):
    delSi = 2*th/(2*N-1)
    z = np.max(x*delSi + 0.5*delSi + bias, 0)
    t = special.gammainc(m/2, z/2)
    return t

In [ ]:
    
#     def objective(params,m,N,E):
#         th, bias = params
#         x=np.ones((len(E),1))*(N-1)-E
#         delSi = 2 * th / (2 * N - 1)
#         z = np.maximum(x * (delSi*np.ones((1,len(E)))) + 0.5 * delSi + bias, 0)
#         t = special.gammainc(m / 2, z / 2)-1
#         return -t  # Negate t since we want to maximize it

In [ ]:
class Optimization():
    
    def __init__(self,params,m,N,Ek):
        self.params=params.T
        self.threshold=params[:,0]
        self.bias=params[:,1]
        self.m=m
        self.N=N
        self.Ek=Ek

    def gradient(self):
        x=np.zeros((len(self.Ek)))
        z=np.zeros((len(self.Ek)))
        t=np.zeros((len(self.Ek)))
        dt_dth=np.zeros((len(self.Ek)))
        dt_dbias=np.zeros((len(self.Ek)))
        for i in range(len(self.Ek)):
            x[i]=self.N-1-self.Ek[i]
            delSi = 2 * self.threshold[i] / (2 * self.N - 1)
            z[i] = np.maximum(x[i] * delSi + 0.5 * delSi + self.bias[i], 0)
            t[i] = 1 - special.gammainc(m / 2, z[i]/ 2)
            dt_dth[i] = 0.5 * z[i] * np.exp(-z[i] / 2) * x[i] / (2 *self.N - 1)
            dt_dbias[i] = 0.5 * z[i] * np.exp(-z[i] / 2)
        return np.array([-dt_dth, -dt_dbias])

    def adam_optimizer(self,learning_rate=0.8, beta1=0.9, beta2=0.999, epsilon=1e-8, num_iterations=2):
        p = np.zeros_like(self.params)
        v = np.zeros_like(self.params)
        initial_values=self.params

        for t in range(1,num_iterations+1):
            gradient = self.gradient()
            p = beta1 * p + (1 - beta1) * gradient
            v = beta2 * v + (1 - beta2) * (gradient ** 2)
            p_hat = p / (1 - beta1 ** t)
            v_hat = v / (1 - beta2 ** t)
            initial_values += learning_rate * p_hat / (np.sqrt(v_hat) + epsilon)

        return initial_values

In [ ]:
import numpy as np

class RLEnvironment():
    def __init__(self, num_states,num_action,cusum,residual,action,pos,N):
        self.num_states = num_states
        self.num_actions = num_action
        self.cusum = cusum
        self.residual = residual
        self.threshold = action[:,0]
        self.absorb=N
        self.current_state = None
        
    def discrete_frame(self):
        delSi = 2*self.threshold/(2*self.absorb-1)
        Ek=np.zeros((len(self.cusum)))
        for i in range(len(self.cusum)):
            if self.cusum[i]<delSi[i]/2:
                Ek[i]=0
            elif self.cusum[i]>=self.threshold[i]:
                Ek[i]=self.absorb
            else :
                Ek[i]=math.ceil(math.floor(self.cusum[i]/(delSi[i]/2))/2)
        return Ek.reshape(1,-1)
        
    def reset(self):
        self.current_state = self.discrete_frame()
        return self.current_state
    
    def calculate_reward(self,i):
        # Calculate reward based on attack detection condition
        if self.residual[i]>0.01 and self.cusum[i]> self.threshold[i]:
            return 10
        elif self.residual[i]<0.01 and self.cusum[i]<self.threshold[i]:
            return 1
        return 0

    def step(self,pos):
        
        next_state = self.discrete_frame()
        
        reward = self.calculate_reward(pos)
        
        self.current_state = next_state
        done = (self.current_state == self.absorb)
        
        return next_state, reward, done,


In [ ]:
import torch
import copy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define actor and critic neural network architectures
class ActorNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(ActorNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.output_layer = nn.Linear(300, action_dim)

    def forward(self, inputs):
        x = F.relu(self.fc1(inputs))
        x = F.relu(self.fc2(x))
        actions = torch.tanh(self.output_layer(x))
        return actions

class CriticNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(CriticNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.output_layer = nn.Linear(300, 1)

    def forward(self, state, action):
        x = torch.cat([state, action[:,0].reshape(1,-1)], dim=-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        q_value = self.output_layer(x)
        return q_value


In [ ]:
# Hyperparameters
threshold = 6
pos=0
BUFFER_SIZE = 100000
BATCH_SIZE = 64
GAMMA = 0.99
TAU = 0.001
LR_ACTOR = 0.0001
LR_CRITIC = 0.001
N=10
threshold=np.ones((len(Sk),1))*6
bias=np.ones((len(Sk),1))*4
action = np.concatenate((threshold,bias),axis=1)
num_action=len(action)
Ek=discrete_frame(Sk,N,action[:,0])
optimizer=Optimization(action,m,N,Ek)
env = RLEnvironment(len(Sk),num_action,Sk,r_mean,action,pos,N)

# Initialize actor, critic, target actor, and target critic networks
state_dim = env.num_states
action_dim = env.num_actions

# Create actor and critic networks
actor = ActorNetwork(state_dim, action_dim)
critic = CriticNetwork(state_dim, action_dim)

# Create target actor and critic networks and initialize them with main network weights
target_actor = copy.deepcopy(actor)
target_critic = copy.deepcopy(critic)

# Define optimizer for actor and critic networks
actor_optimizer = optim.Adam(actor.parameters(), lr=LR_ACTOR)
critic_optimizer = optim.Adam(critic.parameters(), lr=LR_CRITIC)


# Other components like replay buffer, training loop, exploration noise, etc.
# should be implemented and integrated into this code as well.


In [ ]:
action.ravel()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Define other hyperparameters and initialization steps
learning_rate_actor = 0.001
learning_rate_critic = 0.001
learning_rate_threshold = 0.01  # Learning rate for the threshold optimizer
# batch_size = 64
buffer_size = 10000
gamma = 0.99
tau = 0.001
num_episodes = 10
N=10
batch_size=1
for episode in range(num_episodes):
    states = env.reset()
    episode_reward = 0
    done = False
    
    for i in range(len(Sk)):
        # Select action using actor network
        pos=i
        actions = optimizer.adam_optimizer()
        threshold=actions[0]
        bias=actions[1]
        
        
        # Perform action in the environment
        next_states, rewards, dones = env.step(pos)
        print(states)
        print(actions)

        # Convert to PyTorch tensors
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)

        # Update critic network
        q_values = critic(states, actions)
        next_actions = target_actor(next_states)
        next_q_values = target_critic(next_states, next_actions)
        target_q_values = rewards + gamma * (1 - dones) * next_q_values
        critic_loss = nn.MSELoss()(q_values, target_q_values.detach())
        critic_optimizer.zero_grad()
        critic_loss.backward()
        critic_optimizer.step()

        # Update actor network
        actor_loss = -critic(states, actor(states)).mean()
        actor_optimizer.zero_grad()
        actor_loss.backward()
        actor_optimizer.step()
        
        for i in range(1,len(mzk)):
            Sk[i]=max(Sk[i-1]+mzk[i]-bias,0)
        
        states = next_states
        episode_reward += rewards


        # Update target networks using soft updates
        for target_param, param in zip(target_actor.parameters(), actor.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        for target_param, param in zip(target_critic.parameters(), critic.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

    print(f"Episode {episode+1} - Reward: {episode_reward}")

# After training, you can use the trained actor network for generating actions
